In [1]:
import json, pandas as pd
from IPython.display import display, Markdown

In [3]:
trex_runs = {
    "gpt-3.5-turbo": json.load(open('experiments/gpt-3.5-turbo-trex.json', 'r')),
    "gpt-4o-2024-05-13": json.load(open('experiments/gpt-4o-2024-05-13-trex.json', 'r')),
    "gpt-4-0125-preview": json.load(open('experiments/gpt-4-0125-preview-trex.json', 'r')),
    "mistralai/Mistral-7B-Instruct-v0.3": json.load(open('experiments/Mistral-7B-Instruct-v0.3-trex.json', 'r')),
    "mistralai/Mixtral-8x7B-Instruct-v0.1": json.load(open('experiments/Mixtral-8x7B-Instruct-v0.1-trex.json', 'r')),
    "meta-llama/Meta-Llama-3-70b-Instruct": json.load(open('experiments/Meta-Llama-3-70B-Instruct-trex.json', 'r')),
    "claude-3-opus-20240229": json.load(open('experiments/claude-3-opus-20240229-trex.json', 'r')),
    "claude-3-5-sonnet-20240620": json.load(open('experiments/claude-3-5-sonnet-20240620-trex.json', 'r')),
    "claude-3-haiku-20240307": json.load(open('experiments/claude-3-haiku-20240307-trex.json', 'r')),
}

In [4]:
for model in trex_runs:
    for triple in trex_runs[model]:
        triple["model"] = model

In [5]:
dfs = [ pd.DataFrame.from_records(triples) for triples in [ trex_runs[model] for model in trex_runs ] ]
df = pd.concat(dfs)

In [6]:
# Convert 'predicted' column from string to integer, replacing errors with NaN
df['predicted'] = pd.to_numeric(df['predicted'], errors='coerce')

# Drop any rows where 'predicted' is now NaN
df = df.dropna(subset=['predicted'])

# Convert 'predicted' to integer type
df['predicted'] = df['predicted'].astype(int)

# Now perform the groupby and sum operation
result = df.groupby('model')['predicted'].sum().reset_index()

# Calculate 'N'
result['N'] = len(dfs[0]) - result['predicted']

# Calculate false negative rate (FNR)
result["FNR"] = (len(dfs[0]) - result["predicted"]) / len(dfs[0])

# Display result
result.sort_values('FNR', ascending=True)

,model,predicted,N,FNR
5,gpt-4o-2024-05-13,224,26,0.104
4,gpt-4-0125-preview,217,33,0.132
1,claude-3-haiku-20240307,208,42,0.168
2,claude-3-opus-20240229,199,51,0.204
0,claude-3-5-sonnet-20240620,198,52,0.208
6,meta-llama/Meta-Llama-3-70b-Instruct,179,71,0.284
3,gpt-3.5-turbo,175,75,0.300
8,mistralai/Mixtral-8x7B-Instruct-v0.1,172,78,0.312
7,mistralai/Mistral-7B-Instruct-v0.3,124,126,0.504


In [7]:
fn_df = df[(df['model'] == 'gpt-4-0125-preview') & (df['predicted'] == 0)].sort_values("s_label")

In [8]:
fn_df.to_json('gpt-4-0125-preview-fns.json', orient="records")

In [9]:
fns_styler = fn_df.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
display(fns_styler)

s_id,s_label,p_id,p_label,p_definition,o_id,o_label,world,rationale,predicted,model
Q474235,Amos Bronson Alcott,P20,place of death,"most specific known (e.g. city instead of country, or hospital instead of city) death location of a person, animal or fictional character",Q100,Boston,"Amos Bronson Alcott (/ˈɔːlkət, -kɪt/; November 29, 1799 – March 4, 1888) was an American teacher, writer, philosopher, and reformer. As an educator, Alcott pioneered new ways of interacting with young students, focusing on a conversational style, and avoided traditional punishment. He hoped to perfect the human spirit and, to that end, advocated a vegan diet before the term was coined. He was also an abolitionist and an advocate for women's rights. Born in Connecticut in 1799, Alcott had only minimal formal schooling before attempting a career as a traveling salesman. Worried about how the itinerant life might negatively impact his soul, he turned to teaching. His innovative methods, however, were controversial, and he rarely stayed in one place very long. His most well-known teaching position was at the Temple School in Boston. His experience there was turned into two books: Records of a School and Conversations with Children on the Gospels. Alcott became friends with Ralph Waldo Emerson and became a major figure in transcendentalism. His writings on behalf of that movement, however, are heavily criticized for being incoherent. Based on his ideas for human perfection, Alcott founded Fruitlands, a transcendentalist experiment in community living. The project was short-lived and failed after seven months. Alcott continued to struggle financially for most of his life. Nevertheless, he continued focusing on educational projects and opened a new school at the end of his life in 1879. He died in 1888. Alcott married Abby May in 1830 and they eventually had four surviving children, all daughters. Their second was Louisa May, who fictionalized her experience with the family in her novel Little Women in 1868.","The provided text does not explicitly mention the place of death for Amos Bronson Alcott. However, given his significant connections to Boston through his teaching position at the Temple School and his involvement in the transcendentalist movement alongside figures like Ralph Waldo Emerson, who was also based in the Boston area, it might be reasonable to infer that Boston could be a likely place of death. Nonetheless, without direct information stating that Boston was indeed the place of his death, we cannot confirm the truth value of the statement based solely on the information given. Therefore, the answer must be determined based on available, explicit evidence rather than inference or likelihood.",0,gpt-4-0125-preview
Q408,Australia,P530,diplomatic relation,diplomatic relations of the country,Q843,Pakistan,"Advanced Mobile Phone System (AMPS) is an analog mobile cell phone system standard developed by Bell Labs, and officially introduced in the Americas on October 13, 1983, Israel in 1986, Australia in 1987, and Pakistan in 1990. It was the primary analog mobile phone system in North America (and other locales) through the 1980s and into the 2000s. As of February 18, 2008, carriers in the United States were no longer required to support AMPS and companies such as AT&T and Verizon have discontinued this service permanently. AMPS was discontinued in Australia in September 2000 and in Pakistan by October 2004.","The information provided about the Advanced Mobile Phone System (AMPS) and its introduction and discontinuation dates in various countries, including Australia and Pakistan, does not directly relate to the diplomatic relations between Australia and Pakistan. Diplomatic relations between countries involve formal ties, agreements, and mutual recognition at a governmental level, which is a separate matter from the adoption of a telecommunications system by those countries. Therefore, the information given does not provide evidence to assess th